## 4) Oaxaca-Blinder Decomposition

- 선형회귀계수를 이용해 격차를 세 가지 부분으로 분해해 격차의 발생요인을 설명하는 분석 방법

- 가격 격차가 어디서 기인하는지 분석하기 위해 Oaxaca-Blinder 분해법 적용

### Preprocessing

In [182]:
# 읍 X와 y값 지정.
X_eup=log(eup)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_eup=log(eup)['공시지가']

# 면 X와 y값 지정.
X_myun=log(myun)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_myun=log(myun)['공시지가']

# 동 X와 y값 지정.
X_dong=log(dong)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_dong=log(dong)['공시지가']

In [183]:
# 읍, 면 vs 동 Beta(coefficient)계수 벡터화
b_eup = np.asarray(res1.params[1:])
b_myun = np.asarray(res2.params[1:])
b_dong = np.asarray(res3.params[1:])

In [184]:
# 읍, 면 vs 동 독립변수 mean 벡터화
x_eup = np.asarray(X_eup.mean())
x_myun = np.asarray(X_myun.mean())
x_dong = np.asarray(X_dong.mean())

In [185]:
print('읍 공시지가 평균: ',y_eup.mean())
print('면 공시지가 평균: ',y_myun.mean())
print('동 공시지가 평균: ',y_dong.mean())

읍 공시지가 평균:  143.87415273812783
면 공시지가 평균:  68.49730319252787
동 공시지가 평균:  438.9350847985806


### Modeling

- Endowments: 설명 변수 차이로 인해 발생하는 격차

- Coefficients: 각 그룹의 회귀 계수 차이로 인해 발생하는 격차

- Interaction: 모델로 설명할 수 없는 부분 (잔차)

In [186]:
def oaxaca(x1, x2, b1, b2):
    
    X=x1-x2
    B=b1-b2
    
    endowments=[]
    coefficients=[]
    interaction=[]
    
    for i in range(4):
        endowments.append(X[i]*b2[i])
        coefficients.append(x2[i]*B[i])
        interaction.append(X[i]*B[i])
    df = DataFrame()
    df['endowments'] = endowments
    df['coefficents'] = coefficients
    df['interaction'] = interaction
    df['variable'] = ['loglife','logsafety','logtransport','순전입자수']
    df = df.set_index('variable')
    df2 = DataFrame(df.transpose())
    df2['sum'] = df.sum()
    return df2

In [187]:
oaxaca(x_dong, x_myun, b_dong, b_myun)

variable,loglife,logsafety,logtransport,순전입자수,sum
endowments,81.43579,61.90114,50.23192,214.41882,407.98767
coefficents,-107.08869,232.80921,-4.86781,-15.58772,105.26499
interaction,-36.48123,188.28639,-3.07226,-77.26447,71.46844


In [188]:
oaxaca(x_dong, x_eup, b_dong, b_eup)

variable,loglife,logsafety,logtransport,순전입자수,sum
endowments,10.74201,21.58352,12.93237,-107.70444,-62.44654
coefficents,88.87952,354.90958,53.92479,40.64354,538.35743
interaction,15.43369,118.27752,19.79589,249.43507,402.94218
